In [1]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")
raw_datasets

/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [2]:
!jupyter kernelspec list

Available kernels:
  python3        /scratch/qualis/miniconda3/envs/transformer/share/jupyter/kernels/python3
  alpaca         /home01/qualis/.local/share/jupyter/kernels/alpaca
  genai          /home01/qualis/.local/share/jupyter/kernels/genai
  transformer    /home01/qualis/.local/share/jupyter/kernels/transformer


In [3]:
#raw_datasets["train"].select(range(1000)).train_test_split(train_size=0.9, seed=20)

In [4]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)

In [5]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [6]:
split_datasets["validation"] = split_datasets.pop("test")
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [7]:
split_datasets["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

In [8]:
#!pip install sentencepiece

In [9]:
#!pip show sentencepiece

In [10]:
#!pip show transformers
import transformers

Name: transformers
Version: 4.37.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [11]:
#!pip install transformers[all]

In [12]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

[{'translation_text': 'Par défaut pour les threads élargis'}]

In [13]:
split_datasets["train"][172]["translation"]

{'en': 'Unable to import %1 using the OFX importer plugin. This file is not the correct format.',
 'fr': "Impossible d'importer %1 en utilisant le module d'extension d'importation OFX. Ce fichier n'a pas un format correct."}

In [14]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

[{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}]

In [15]:
for text in raw_datasets["train"]["translation"]:
    if " email" in text["en"]:
        print(f"{text}\n")
        print(translator(text["en"]))
        break

{'en': "Otherwise, forward us the backtraces when the email pops-up (you'll see). If you get a crash often you will get it fixed for sure if you forward a really good backtrace to us. See the next sections for assistance there.", 'fr': 'Sinon, envoyez -nous les logs quand le courrier électronique apparaît (vous verrez). Si & amarok; plante souvent, le bogue sera corrigé si vous nous envoyez un bon log. Voir la section suivante pour savoir comment obtenir un bon log.'}

[{'translation_text': "Sinon, faites-nous suivre les backtraces lorsque l'email s'affiche (vous verrez). Si vous obtenez souvent un crash, vous l'obtiendrez corrigé pour être sûr que vous nous transmettez une très bonne backtrace. Consultez les prochaines sections pour de l'aide là-bas."}]


In [16]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")


In [17]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence)
with tokenizer.as_target_tokenizer():
    targets = tokenizer(fr_sentence)

/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))

['▁Default', '▁to', '▁expanded', '▁thread', 's', '</s>']


In [19]:
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [20]:
wrong_targets = tokenizer(fr_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [21]:
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [22]:
targets

{'input_ids': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [24]:
wrong_targets = tokenizer(fr_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [25]:
with tokenizer.as_target_tokenizer():
    target = tokenizer(fr_sentence)

print(target)

{'input_ids': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [26]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-fr', vocab_size=59514, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	59513: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [27]:
max_input_length = 128
model_inputs = tokenizer(en_sentence, max_length=max_input_length, truncation=True)
print(f"input_ids: {model_inputs['input_ids']}")
print(f"text: {tokenizer.decode(model_inputs['input_ids'])}")
print(f"tokens: {tokenizer.convert_ids_to_tokens(model_inputs['input_ids'])}")

input_ids: [47591, 12, 9842, 19634, 9, 0]
text: Default to expanded threads</s>
tokens: ['▁Default', '▁to', '▁expanded', '▁thread', 's', '</s>']


In [28]:
max_input_length = 128
model_inputs = tokenizer(en_sentence, max_length=max_input_length, padding = "max_length", truncation=True) # len(input_ids): 128
model_inputs = tokenizer(en_sentence, max_length=max_input_length, padding = True, truncation=True) #len(input_ids):6  
print(f"input_ids: {model_inputs['input_ids']}")
print(f"text: {tokenizer.decode(model_inputs['input_ids'])}")
print(f"tokens: {tokenizer.convert_ids_to_tokens(model_inputs['input_ids'])}")

input_ids: [47591, 12, 9842, 19634, 9, 0]
text: Default to expanded threads</s>
tokens: ['▁Default', '▁to', '▁expanded', '▁thread', 's', '</s>']


In [29]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-fr', vocab_size=59514, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	59513: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [30]:
split_datasets["train"][1]

{'id': '152754',
 'translation': {'en': 'Default to expanded threads',
  'fr': 'Par défaut, développer les fils de discussion'}}

In [31]:
max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    #print(f"examples: {examples['translation']}")
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    #print(f"labels: {labels}")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [32]:
max_length = 128

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
        #inputs, text_target=targets, max_length=max_length, padding=True, truncation=True
    )
    return model_inputs

In [33]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

In [34]:
print(tokenized_datasets["train"][:2])
print([len(elem) for elem in tokenized_datasets["train"][:10]["input_ids"]])

{'input_ids': [[34378, 226, 5783, 32, 200, 12, 3647, 4, 1223, 1628, 117, 4923, 23608, 3, 1789, 2942, 20059, 301, 548, 301, 331, 30, 117, 4923, 12, 4, 1528, 668, 3, 5734, 212, 9319, 30, 4, 4923, 57, 5487, 30, 4, 6, 32712, 25, 7243, 1160, 12, 621, 42, 4, 1156, 3009, 3, 0], [47591, 12, 9842, 19634, 9, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]], 'labels': [[60, 7418, 5244, 8234, 740, 4993, 8, 6471, 5, 2218, 29, 193, 2220, 742, 3, 4366, 14237, 14, 6, 16600, 301, 548, 301, 331, 5, 193, 24275, 17, 8, 668, 6142, 3, 33640, 36, 81, 6, 5411, 2709, 9376, 22, 24275, 59, 36, 19, 9376, 153, 402, 29033, 13774, 402, 29033, 416, 27, 8, 4034, 4888, 3, 0], [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]]}
[52, 6, 15, 3, 12, 7, 128, 59, 3, 4]


In [35]:
inputs = preprocess_function(split_datasets["train"][:2])
inputs

{'input_ids': [[34378, 226, 5783, 32, 200, 12, 3647, 4, 1223, 1628, 117, 4923, 23608, 3, 1789, 2942, 20059, 301, 548, 301, 331, 30, 117, 4923, 12, 4, 1528, 668, 3, 5734, 212, 9319, 30, 4, 4923, 57, 5487, 30, 4, 6, 32712, 25, 7243, 1160, 12, 621, 42, 4, 1156, 3009, 3, 0], [47591, 12, 9842, 19634, 9, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]], 'labels': [[60, 7418, 5244, 8234, 740, 4993, 8, 6471, 5, 2218, 29, 193, 2220, 742, 3, 4366, 14237, 14, 6, 16600, 301, 548, 301, 331, 5, 193, 24275, 17, 8, 668, 6142, 3, 33640, 36, 81, 6, 5411, 2709, 9376, 22, 24275, 59, 36, 19, 9376, 153, 402, 29033, 13774, 402, 29033, 416, 27, 8, 4034, 4888, 3, 0], [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]]}

In [36]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [37]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(59514, 512, padding_idx=59513)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(59514, 512, padding_idx=59513)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

### 데이터 콜레이션 (Data Collation)
동적 배치 처리(Dynamic batching)를 위한 패딩을 처리하려면 데이터 콜레이터가 필요합니다. 이 경우 3장에서와 같이 DataCollatorWithPadding을 사용할 수 없습니다. 왜냐하면 이 메서드는 입력(input IDs, attention mask 및 token type IDs)에 대해서만 패딩을 수행하기 때문입니다. 레이블 역시 레이블에 있는 최대 길이로 채워져야 합니다. 그리고 앞서 언급했듯이 레이블을 채우는데 사용되는 패딩 값은 토크나이저의 패딩 토큰이 아니라 -100이어야 합니다. 그래야 패딩된 값이 손실 계산에서 무시됩니다.

이 작업은 모두 DataCollatorForSeq2Seq에 의해 수행됩니다. DataCollatorWithPadding과 마찬가지로, DataCollatorForSeq2Seq는 입력을 전처리하는데 사용되는 tokenizer는 물론 모델 자체도 매개변수로 입력받습니다. 모델도 입력받는 이유는 이 데이터 콜레이터가 시작 부분에 특수 토큰이 붙어 있는, 레이블 시퀀스를 우측으로 시프트(shift)한 버전인 디코더 input IDs를 준비하는 역할도 하기 때문입니**다. 이 시프트(shift) 방법은 아키텍처마다 약간씩 다르기 때문에 DataCollatorForSeq2Seq는 모델 객체를 알아야 합**.l)


In [38]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [39]:
data_collator_inputs = [tokenized_datasets["train"][i] for i in range(1, 3)]
print(data_collator_inputs)

[{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}, {'input_ids': [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 28149, 139, 33712, 25218, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]}]


In [40]:
#data_collator_inputs = [tokenized_datasets["train"][i] for i in range(1, 3)]
#print(data_collator_inputs)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)]) # return batched tensor
batch


{'input_ids': tensor([[47591,    12,  9842, 19634,     9,     0, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
         28149,   139, 33712, 25218,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]]), 'decoder_input_ids': tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])}

In [41]:
# note the shift in the decoder_input_ids 
print(batch["input_ids"][:1]) # 59513 is padded in the input_ids
print(batch["labels"][:1]) # note that -100 is padded in the labels, i.e. decoder target
print(batch["decoder_input_ids"][:1]) # 59513 is padded in the decode inputs  

tensor([[47591,    12,  9842, 19634,     9,     0, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513]])
tensor([[ 577, 5891,    2, 3184,   16, 2542,    5, 1710,    0, -100, -100, -100,
         -100, -100, -100, -100]])
tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513]])


In [42]:
print([tokenized_datasets["train"][i] for i in range(1, 3)])

[{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}, {'input_ids': [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 28149, 139, 33712, 25218, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]}]


In [43]:
raw_datasets["train"]["translation"][1:3]

[{'en': '& Lauri. Watts. mail;', 'fr': '& Lauri. Watts. mail;'},
 {'en': 'ROLES_OF_TRANSLATORS', 'fr': '& traducteurJeromeBlanc;'}]

In [44]:
sample_datasets = tokenized_datasets["train"][1:3]
print(sample_datasets)
sample_datasets_list = [sample_datasets]
print(sample_datasets_list)
print({k: sum(sample_datasets[k], []) for k in sample_datasets.keys()})
#print(sample_datasets[0])
print(tokenized_datasets["train"][2])
print("*sample_datasets:", *sample_datasets)
print("*sample_datasets.values(): ", *sample_datasets.values())
for t in zip(*sample_datasets.values()):
    print(t)
print([dict(zip(sample_datasets, t)) for t in zip(*sample_datasets.values())])

{'input_ids': [[47591, 12, 9842, 19634, 9, 0], [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 28149, 139, 33712, 25218, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0], [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]]}
[{'input_ids': [[47591, 12, 9842, 19634, 9, 0], [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 28149, 139, 33712, 25218, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0], [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]]}]
{'input_ids': [47591, 12, 9842, 19634, 9, 0, 1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 28149, 139, 33712, 25218, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0, 1211, 3, 49, 9409,

In [45]:
import numpy as np
np.array(batch["labels"])

array([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         -100,  -100,  -100,  -100,  -100,  -100,  -100],
       [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
          817,   550,  7032,  5821,  7907, 12649,     0]])

In [46]:
batch["decoder_input_ids"]

tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])

In [47]:
print("decoder input: ", tokenizer.decode([59513,   577,  5891,  2,  3184,  16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513]))
print("decoder output: ", tokenizer.decode([  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100]))

decoder input:  <pad> Par défaut, développer les fils de discussion</s> <pad> <pad> <pad> <pad> <pad> <pad>
decoder output:  Par défaut, développer les fils de discussion</s> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


In [48]:
print("decoder input: ", tokenizer.decode([59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]))
print("decoder output: ", tokenizer.decode([ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
          817,   550,  7032,  5821,  7907, 12649,     0]))

decoder input:  <pad> *. ui *. UI_BAR_Fichiers interface utilisateur
decoder output:  *. ui *. UI_BAR_Fichiers interface utilisateur</s>


In [49]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-fr', vocab_size=59514, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	59513: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [50]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]
[1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]


In [51]:
#!python3 -m pip install sacrebleu

In [52]:
from datasets import load_metric

metric = load_metric("sacrebleu")

/tmp/ipykernel_35207/200089639.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [53]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]

references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)


{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [54]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)


{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [55]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)


{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

In [56]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-fr', vocab_size=59514, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	59513: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [57]:
"""
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")
#split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets = raw_datasets["train"].select(range(10000)).train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

print(tokenized_datasets)
#DatasetDict({
#    train: Dataset({
#        features: ['input_ids', 'attention_mask', 'labels'],
#        num_rows: 189155
#    })
#    validation: Dataset({
#        features: ['input_ids', 'attention_mask', 'labels'],
#        num_rows: 21018
#    })
#})

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

from torch.utils.data import DataLoader

#tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], 
    collate_fn=data_collator, 
    batch_size=8
)

optimizer = AdamW(model.parameters(), lr=2e-5)

for batch in train_dataloader:
    break
print(batch.keys()) #dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])
#print(batch)

#outputs = model(**batch) 
#print(outputs.logits.shape) #torch.Size([8, 39, 59514])
#print(outputs.loss)

#predictions = outputs.logits.argmax(-1) 
#print(f"predictions shape: {predictions.shape}") #torch.Size([8, 39])
#print(f"preds: {predictions}")
#print(f"labels: {batch['labels']}")

#decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#print(decoded_preds)

#print(tokenizer.decode(predictions[0].tolist()))
#output = outputs.logits[0,:,:]
#print(output.shape)
#tokenizer.decode

import numpy as np

def compute_metrics(preds, labels):
    print(labels)
    #preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우.
    if isinstance(preds, tuple):
        preds = preds[0]

    #print(f"prediction: {preds.shape}, {preds}") #torch.Size([8, 39]), batch size == 8
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    print(decoded_preds)

    #print(f"labels : {labels.shape}, {type(labels)}") #torch.Size([8, 39]), batch size == 8
    # -100은 건너뛴다.
    #labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)

    #print(labels)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    print(decoded_labels)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    print(decoded_preds)
    print(decoded_labels)
## decoded_preds
#["L'éditeur d'entrée pour pour les champs de L L L L", 'Taille du cylindre Taille Taille 
#Taille Taille Taille Taille Taille Taille Taille Taille Taille Taille', 'Allemandvieen 
#deryerenAllemagnehé) président le raisonss de la #ids.', 'La fenêtre de marche La La La La La 
#La La La La La La', 'Ctrl; P Fichier Imprimer... C C C C C C C C', 'Le mode Veurs de sources; 
#V VPL.. Le Le Le', 'Utilisation de & kkuickshow; Utilisation Utilisation Utilisation', 
#'Types de champs Types Types Types Types Types Types Types Types Types Types Types Types 
#Types Types']
## decoded_labels    
#[["L'éditeur d'entrées pour les champs Tableau"], ['Taille de cylindre'], 
#['Jan Willem van de Meent (Adios), pour les icônes de & krusader;'], ['La fenêtre de progression'],
#['Ctrl; P Fichier Imprimer...'], ['Le mode Éditeur de source et & VPL;.'],
#['Utilisation de & kuickshow;'], ['Types de champs']]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


print(batch["labels"])

scores = compute_metrics(predictions, batch["labels"])
print(scores)
"""

'\nimport torch\nfrom transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification\n\nfrom datasets import load_dataset, load_metric\n\nraw_datasets = load_dataset("kde4", lang1="en", lang2="fr")\n#split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)\nsplit_datasets = raw_datasets["train"].select(range(10000)).train_test_split(train_size=0.9, seed=20)\nsplit_datasets["validation"] = split_datasets.pop("test")\n\nmodel_checkpoint = "Helsinki-NLP/opus-mt-en-fr"\n#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")\ntokenizer = AutoTokenizer.from_pretrained(model_checkpoint)\n\nmax_input_length = 128\nmax_target_length = 128\n\n\ndef preprocess_function(examples):\n    inputs = [ex["en"] for ex in examples["translation"]]\n    targets = [ex["fr"] for ex in examples["translation"]]\n    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)\n\n    # 타겟을 위한 토크나이저 셋업\n    with tokenizer.as_target

In [58]:
#max_values, max_indices = torch.max(output, dim=1)
#print(max_values)
#print(max_indices.tolist())
#print(tokenizer.convert_ids_to_tokens(max_indices.tolist()))
#print(tokenizer.decode(max_indices.tolist()))
#print(tokenizer.decode(max_indices.tolist(), skip_special_tokens=True))
#print(batch["labels"][0])

In [59]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우.
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


In [60]:
#from huggingface_hub import notebook_login

#notebook_login()


In [61]:
#!pip install accelerate -U

In [62]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    #push_to_hub=True,
)


In [63]:
from transformers import Seq2SeqTrainer
print(tokenized_datasets["train"])
print(tokenized_datasets["validation"])

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 189155
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 21018
})


In [64]:
max_target_length

128

In [65]:
import torch
torch.cuda.is_available()

True

In [66]:
trainer.evaluate(max_length=max_target_length)

/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.6985208988189697,
 'eval_bleu': 39.27124165416069,
 'eval_runtime': 534.5806,
 'eval_samples_per_second': 39.317,
 'eval_steps_per_second': 0.155}

In [67]:
trainer.train()

Step,Training Loss
500,1.318500
1000,1.139300
1500,1.067200
2000,0.999800
2500,0.976100
3000,0.958000
3500,0.921400
4000,0.915100


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}
/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception 

TrainOutput(global_step=4434, training_loss=1.0253028990074917, metrics={'train_runtime': 769.6242, 'train_samples_per_second': 737.327, 'train_steps_per_second': 5.761, 'total_flos': 1.6505690950139904e+16, 'train_loss': 1.0253028990074917, 'epoch': 3.0})

In [ ]:
trainer.evaluate(max_length=max_target_length)

/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
#trainer.push_to_hub(tags="tanslation", commit_message="Training complete")

In [ ]:
from transformers import pipeline

model_checkpoint = "hwang2006/marian-finetuned-kde4-en-to-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")


In [ ]:
#!pip install sacremoses

In [ ]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)


In [ ]:
for text in raw_datasets["train"]["translation"]:
    if " email" in text["en"]:
        print(f"{text}\n")
        print(translator(text["en"]))
        break

### 
맞춤형 학습 루프 (Custom Training Loop
)
이제 전체 학습 루프(full training loop)를 살펴보고 필요한 부분을 쉽게 커스터마이징할 수 있습니다. 7.2와 7.3에서 했던 것과 매우 유사할 것입니다.

학습을 위한 모든 사항 준비하기
아래 내용은 이미 몇번씩 공부했으므로 코드를 매우 빠르게 살펴보겠습니다. 먼저 데이터셋을 "torch" 형식으로 설정한 후 데이터셋에서 DataLoader를 빌드하여 PyTorch 텐서를 얻습니다:

In [ ]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], 
    collate_fn=data_collator, 
    batch_size=8
)


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
#from huggingface_hub import Repository, get_full_repo_name

#model_name = "marian-finetuned-kde4-en-to-fr-accelerate"
#repo_name = get_full_repo_name(model_name)
#repo_name

In [ ]:
#output_dir = "marian-finetuned-kde4-en-to-fr-accelerate"
#repo = Repository(output_dir, clone_from=repo_name)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels


In [ ]:
from tqdm.auto import tqdm
import torch
import numpy as np

output_dir = "marian-finetuned-kde4-en-to-fr-accelerate"

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # 학습
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # 평가
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # 예측과 레이블을 모으기 전에 함께 패딩 수행
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # 저장 및 업로드
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        #repo.push_to_hub(
        #    commit_message=f"Training in progress epoch {epoch}", blocking=False
        #)


In [1]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")
#raw_datasets = raw_datasets["train"].select(range(1000))
#split_datasets = raw_datasets["train"].select(range(10000)).train_test_split(train_size=0.90, seed=20)
split_datasets = raw_datasets["train"].train_test_split(train_size=0.95, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

print(split_datasets)

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

#print(tokenized_datasets)
#DatasetDict({
#    train: Dataset({
#        features: ['input_ids', 'attention_mask', 'labels'],
#        num_rows: 189155
#    })
#    validation: Dataset({
#        features: ['input_ids', 'attention_mask', 'labels'],
#        num_rows: 21018
#    })
#})

from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

import numpy as np

from datasets import load_metric
metric = load_metric("sacrebleu")


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    #print(preds.shape, labels.shape) # (500, 512) (500, 512)
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우.
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # replace -100 with pad_token
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    #evaluation_strategy="no",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    #per_device_train_batch_size=32,
    #per_device_eval_batch_size=64,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    #push_to_hub=True,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print(f"before training: {trainer.evaluate(max_length=max_target_length)}")

trainer.train()

print(f"after training: {trainer.evaluate(max_length=max_target_length)}")



/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 199664
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 10509
    })
})


/tmp/ipykernel_6575/850021938.py:61: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Acceler

before training: {'eval_loss': 1.7459030151367188, 'eval_bleu': 38.9906643720949, 'eval_runtime': 309.8045, 'eval_samples_per_second': 33.921, 'eval_steps_per_second': 1.062}


Epoch,Training Loss,Validation Loss,Bleu
1,0.995100,0.924919,49.642413
2,0.891300,0.869644,52.180058
3,0.827100,0.854542,52.685193


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}
/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception 

after training: {'eval_loss': 0.8545421957969666, 'eval_bleu': 52.64871528841124, 'eval_runtime': 327.0346, 'eval_samples_per_second': 32.134, 'eval_steps_per_second': 1.006, 'epoch': 3.0}


In [2]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSeq2SeqLM

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")
#split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets = raw_datasets["train"].shuffle(seed=20).select(range(10000)).train_test_split(train_size=0.95, seed=20)
split_datasets = raw_datasets["train"].train_test_split(train_size=0.95, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

print(tokenized_datasets)
#DatasetDict({
#    train: Dataset({
#        features: ['input_ids', 'attention_mask', 'labels'],
#        num_rows: 189155
#    })
#    validation: Dataset({
#        features: ['input_ids', 'attention_mask', 'labels'],
#        num_rows: 21018
#    })
#})

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

from torch.utils.data import DataLoader

#tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], 
    collate_fn=data_collator, 
    batch_size=8
)

optimizer = AdamW(model.parameters(), lr=2e-5)

from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

#from huggingface_hub import Repository, get_full_repo_name

#model_name = "marian-finetuned-kde4-en-to-fr-accelerate"
#repo_name = get_full_repo_name(model_name)
#repo_name

output_dir = "marian-finetuned-kde4-en-to-fr-accelerate"
#repo = Repository(output_dir, clone_from=repo_name)

def postprocess(predictions, labels):
    #print(f"postprocess: {predictions.shape}, {labels.shape}")
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

from datasets import load_metric
metric = load_metric("sacrebleu")

from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # 학습
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # 평가
    model.eval()
    for batch in tqdm(eval_dataloader):
    #for batch in eval_dataloader:
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        #print(generated_tokens.shape, labels.shape)

        # 예측과 레이블을 모으기 전에 함께 패딩 수행
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # 저장 및 업로드
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        #repo.push_to_hub(
        #    commit_message=f"Training in progress epoch {epoch}", blocking=False
        #)



/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 199664
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10509
    })
})


/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/74874 [00:00<?, ?it/s]

  0%|          | 0/1314 [00:00<?, ?it/s]

...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}


epoch 1, BLEU score: 53.02


  0%|          | 0/1314 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}


epoch 2, BLEU score: 53.90


In [ ]:
#!pip install sacremoses

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

from transformers import AdamW # try out with torch.optim.AdamW
#from torch.optim import AdamW

from datasets import load_dataset, load_metric

from torch.utils.data import DataLoader
from accelerate import Accelerator
from transformers import get_scheduler

import evaluate

from tqdm.auto import tqdm

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")
#split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets = raw_datasets["train"].shuffle(seed=20).select(range(10000)).train_test_split(train_size=0.95, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"

max_input_length = 128
max_target_length = 128

def get_dataloaders(accelerator: Accelerator, model, batch_size: int = 32):

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    max_input_length = 128
    max_target_length = 128


    def preprocess_function(examples):
        inputs = [ex["en"] for ex in examples["translation"]]
        targets = [ex["fr"] for ex in examples["translation"]]
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

        # 타겟을 위한 토크나이저 셋업
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, truncation=True)

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # Apply the method we just defined to all the examples in all the splits of the dataset
    # starting with the main process first:
    with accelerator.main_process_first():
        tokenized_datasets = split_datasets.map(
            preprocess_function,
            batched=True,
            remove_columns=split_datasets["train"].column_names,
        )

    #print(tokenized_datasets)
    #DatasetDict({
    #    train: Dataset({
    #        features: ['input_ids', 'attention_mask', 'labels'],
    #        num_rows: 189155
    #    })
    #    validation: Dataset({
    #        features: ['input_ids', 'attention_mask', 'labels'],
    #        num_rows: 21018
    #    })
    #})

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    #tokenized_datasets.set_format("torch")
    train_dataloader = DataLoader(
        tokenized_datasets["train"],
        shuffle=True,
        collate_fn=data_collator,
        batch_size=batch_size,
    )
    eval_dataloader = DataLoader(
        tokenized_datasets["validation"], 
        collate_fn=data_collator, 
        batch_size=batch_size,
    )

    return train_dataloader, eval_dataloader, tokenizer


#def training_function(config, args):
def training_function():

    batch_size = 8

    accelerator = Accelerator()
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
    train_dataloader, eval_dataloader, tokenizer = get_dataloaders(accelerator, model, batch_size=batch_size)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
    optimizer = AdamW(model.parameters(), lr=2e-5)

    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader
    )

    num_train_epochs = 3
    num_update_steps_per_epoch = len(train_dataloader)
    num_training_steps = num_train_epochs * num_update_steps_per_epoch

    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    output_dir = "marian-finetuned-kde4-en-to-fr-accelerate"

    def postprocess(predictions, labels):
        #print(f"postprocess: {predictions.shape}, {labels.shape}")
        predictions = predictions.cpu().numpy()
        labels = labels.cpu().numpy()

        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

        # Replace -100 in the labels as we can't decode them
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        return decoded_preds, decoded_labels

    #metric = load_metric("sacrebleu")
    metric = evaluate.load("sacrebleu")

    progress_bar = tqdm(range(num_training_steps))

    for epoch in range(num_train_epochs):
        # 학습
        model.train()
        for batch in train_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

        # 평가
        model.eval()
        for batch in tqdm(eval_dataloader):
        #for batch in eval_dataloader:
            with torch.no_grad():
                generated_tokens = accelerator.unwrap_model(model).generate(
                    batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    max_length=128,
                )
            labels = batch["labels"]

            #print(generated_tokens.shape, labels.shape)
            #torch.Size([8, 15]) torch.Size([8, 17])
            #torch.Size([8, 13]) torch.Size([8, 12])
            #torch.Size([8, 32]) torch.Size([8, 28])
            #torch.Size([8, 61]) torch.Size([8, 59])

            # 예측과 레이블을 모으기 전에 함께 패딩 수행
            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

            #print(f"after padding: {generated_tokens.shape}, {labels.shape}")
            #after padding: torch.Size([8, 61]), torch.Size([8, 59])
            #after padding: torch.Size([8, 61]), torch.Size([8, 59])
            #after padding: torch.Size([8, 61]), torch.Size([8, 59])
            #after padding: torch.Size([8, 61]), torch.Size([8, 59])
            

            predictions_gathered = accelerator.gather(generated_tokens)
            labels_gathered = accelerator.gather(labels)

            #print(f"after gathered: {predictions_gathered.shape}, {labels_gathered.shape}")
            #after gathered: torch.Size([32, 61]), torch.Size([32, 59])
            #after gathered: torch.Size([32, 61]), torch.Size([32, 59])
            #after gathered: torch.Size([32, 61]), torch.Size([32, 59])
            #after gathered: torch.Size([32, 61]), torch.Size([32, 59])

            decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
            metric.add_batch(predictions=decoded_preds, references=decoded_labels)

        results = metric.compute()
        #print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")
        accelerator.print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

        # 저장 및 업로드
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
        if accelerator.is_main_process:
            tokenizer.save_pretrained(output_dir)
            #repo.push_to_hub(
            #    commit_message=f"Training in progress epoch {epoch}", blocking=False
            #)

#from huggingface_hub import Repository, get_full_repo_name

#model_name = "marian-finetuned-kde4-en-to-fr-accelerate"
#repo_name = get_full_repo_name(model_name)
#repo_name
#repo = Repository(output_dir, clone_from=repo_name)
#notebook_launcher(function, args, num_processes, mixed_precision, use_port, master_addr, node_rank, num_nodes)

from accelerate import notebook_launcher

notebook_launcher(training_function, num_processes=4)
    